In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

### This is a playground!

Here you can quickly test some code, visualize your data

In [10]:
import torch
import torch.optim as optim
import torch.nn as nn
from Project import Project
from data.DetectionDataset import DetectionDataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from utils import device

device

'cuda'

In [11]:
pr = Project()

yoda_dir = pr.data_dir / 'yoda'

annotations_path = list(yoda_dir.glob('*.txt'))
images_path = list(yoda_dir.glob('*.jpg'))

class ImgBBoxsTr():
    def __init__(self):
        self.to_tensor = ToTensor()
        
    def __call__(self, x, bboxs):
        return self.to_tensor(x).float(), torch.Tensor(bboxs)

In [13]:
ds = DetectionDataset(images_path, annotations_path, transform=ImgBBoxsTr())

ds[1]

(tensor([[[0.0706, 0.0706, 0.0706,  ..., 0.0627, 0.0627, 0.0627],
          [0.0706, 0.0706, 0.0706,  ..., 0.0627, 0.0627, 0.0627],
          [0.0706, 0.0706, 0.0706,  ..., 0.0627, 0.0627, 0.0627],
          ...,
          [0.1020, 0.1020, 0.1020,  ..., 0.1020, 0.1020, 0.1020],
          [0.1020, 0.1020, 0.1020,  ..., 0.1020, 0.1020, 0.1020],
          [0.1020, 0.1020, 0.1020,  ..., 0.1020, 0.1020, 0.1020]],
 
         [[0.0431, 0.0431, 0.0431,  ..., 0.0275, 0.0275, 0.0275],
          [0.0431, 0.0431, 0.0431,  ..., 0.0275, 0.0275, 0.0275],
          [0.0431, 0.0431, 0.0431,  ..., 0.0275, 0.0275, 0.0275],
          ...,
          [0.0627, 0.0627, 0.0627,  ..., 0.0784, 0.0784, 0.0784],
          [0.0627, 0.0627, 0.0627,  ..., 0.0784, 0.0784, 0.0784],
          [0.0627, 0.0627, 0.0627,  ..., 0.0784, 0.0784, 0.0784]],
 
         [[0.0039, 0.0039, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ..., 0.0000, 0.0000, 0.0000],
          [0.0039, 0.0039, 0.0039,  ...,

In [14]:
dl = DataLoader(ds, batch_size=4, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

## Model

In [15]:
??fasterrcnn_resnet50_fpn

In [16]:
from torchvision.models.detection.faster_rcnn  import fasterrcnn_resnet50_fpn

model = fasterrcnn_resnet50_fpn(num_classes=2).to(device)


## Training

In [17]:
# https://github.com/pytorch/vision/blob/master/references/detection/engine.py
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

In [18]:
for i in range(100):

    for images, targets in dl:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k,v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        print(losses)

tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)
tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)
tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)
tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)
tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)
tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)
tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 